# related papers
http://journals.sagepub.com/doi/pdf/10.1177/0081175015576601

http://www.lifescienceglobal.com/pms/index.php/ijcs/article/view/5052/2853

https://papers.ssrn.com/sol3/papers.cfm?abstract_id=2770506

https://www.ijdp.org/article/S0955-3959(17)30288-8/abstract

http://article.socialsciencesjournal.org/pdf/10.11648.j.ss.20180702.15.pdf

https://www.sciencedirect.com/science/article/pii/S0091743503002330

https://www.sciencedirect.com/science/article/pii/S0013935109000498

https://onlinelibrary.wiley.com/doi/abs/10.1111/ecpo.12102

http://journals.sagepub.com/doi/abs/10.1177/0003122416663494

http://journals.sagepub.com/doi/pdf/10.1177/0022427806298356

https://waset.org/abstracts/environmental-and-ecological-engineering/76986

http://www.mdpi.com/2220-9964/7/2/43

http://discovery.ucl.ac.uk/11291/1/11291.pdf (hot spot report)

In [1]:
import pandas as pd
from shapely.geometry import Point
import geopandas as gp

import datetime
from dateutil.relativedelta import relativedelta


from src import constants as C
from src.data_prep import prep_data_from_raw, prep_911, prep_crime

# load data

In [2]:
d911 = prep_911(by_category=False, gpdf=True, coords_series=False)
d911.reset_index(inplace=True)
d911.crs

{'init': 'epsg:3559', 'no_defs': True}

In [3]:
crimes = prep_crime(path='data/open-baltimore/clean/crimes-dev-set.csv',by_category=False, gpdf=True, coords_series=False)
crimes.reset_index(inplace=True)

In [4]:
# crimes = prep_data_from_raw('data/open-baltimore/raw/BPD_Part_1_Victim_Based_Crime_Data.csv', 
#                  col_lon='Longitude', col_lat='Latitude', col_date='CrimeDate', to_epsg=3559)

# crimes= crimes.loc['2015-01-01':]
# crimes['geometry'] = crimes.Coords.apply(lambda x: Point(*x))
# crimes = gp.GeoDataFrame(crimes)
# crimes.geometry = crimes.buffer(30)
# crimes.reset_index(inplace=True)
# crimes.crs = {'init': 'epsg:3559', 'no_defs': True}

# sjoin

In [5]:
# do sjoin per (Year, Month)
ym_crime = set(crimes.DateTime.apply(lambda x: (x.year, x.month)).unique())
ym_911 = set(d911.DateTime.apply(lambda x: (x.year, x.month)).unique())
yms = list(ym_crime & ym_911)
len(yms)

24

In [6]:
# sjoin within 100 meters
# d911.geometry = d911.buffer(30)
crimes.geometry = crimes.buffer(100)

In [7]:
pairs = []
for y, m in yms:
    # - 1 day in case a crime happened just seconds before 1st of a month
    # include the 1st of next month in case a crime happend just seconds after this month
    # this may create duplicates, drop duplicates later
    date = datetime.datetime(year=y, month=m, day=1)
    start = date - datetime.timedelta(days=1)
    end = date + relativedelta(months=+1)
    
    # slice data by start and end date
    sted_crimes = crimes[(crimes.DateTime>=start) & (crimes.DateTime<=end)]
    sted_d911 = d911[(d911.DateTime>=start) & (d911.DateTime<=end)]
    
    # Spatial constraint: 911 call is within X-meter radius of crime
    joined = gp.sjoin(sted_d911[['DateTime', 'geometry']], sted_crimes[['DateTime', 'geometry']])
    
    # DateTime constraint: 911 call is within 12 hours after crime happened, 
    # or 911 can also be within 1 hour before crime, considering the crimeTime may not be accurate
    joined = joined.reset_index().rename(columns={'index': 'i911', 'index_right': 'icrime'})
    joined['dt_diff'] = joined.apply(lambda x: (x.DateTime_left - x.DateTime_right).total_seconds()/3600, axis=1)
    cond = (joined.dt_diff >= -1) & (joined.dt_diff <= 12 )
    pairs.append(joined[cond][['i911', 'icrime', 'dt_diff']])
#     break

pairs = pd.concat(pairs, ignore_index=True)
# drop duplicates
pairs = pairs[~pairs.duplicated()]

In [8]:
num_crimes = crimes[(crimes.DateTime>='2015-01-01') & (crimes.DateTime<='2017-01-01')].shape[0]
num_911 = d911[(d911.DateTime>='2015-01-01') & (d911.DateTime<='2017-01-01')].shape[0]

In [9]:
idx_crimes = pairs.icrime.unique()
idx_911 = pairs.i911.unique()

In [10]:
'crime: %d/%d=%.02f, 911: %d/%d=%.02f' % (len(idx_crimes), num_crimes,len(idx_crimes) / num_crimes, len(idx_911), num_911, len(idx_911) / num_911)

'crime: 49714/92820=0.54, 911: 72460/634100=0.11'

# add info to pairs

In [11]:
# information to be added to pairs

col_c = ['DateTime', 'Category', 'Latitude', 'Longitude', 'Weapon', 'Neighborhood', 'In/Outside', 'Coords']
target_c = crimes.loc[idx_crimes][col_c]
col_c = [c+'_c' for c in col_c]
target_c.columns = col_c

col_9 = ['DateTime', 'Category', 'description', 'Latitude', 'Longitude', 'priority', 'Coords']
target_9 = d911.loc[idx_911][col_9]
col_9 = [c+'_9' for c in col_9]
target_9.columns = col_9

In [12]:
# for do-over merge
# pairs = pairs[['i911', 'icrime', 'dt_diff', 'distance']]

In [13]:
pairs = pairs.merge(target_9, left_on='i911', right_index=True).merge(target_c, left_on='icrime', right_index=True)

In [ ]:
from scipy.spatial import distance
pairs['distance'] = pairs.apply(lambda x: distance.euclidean(x.Coords_9, x.Coords_c), axis=1)

In [18]:
pairs.set_index(['icrime', 'Category_c', 'DateTime_c','i911']).sort_index()

dt_diff  \
icrime Category_c    DateTime_c          i911                
91050  theft_larceny 2014-12-31 15:20:00 83      10.216667   
91051  robbery       2014-12-31 15:00:00 148     11.633333   
91056  assault       2014-12-31 21:30:00 90       4.133333   
                                         104      4.400000   
91060  theft_larceny 2014-12-31 23:30:00 239      5.050000   
                                         247      5.150000   
                                         306      6.566667   
                                         318      7.100000   
91063  theft_larceny 2014-12-31 23:00:00 420     11.816667   
91064  theft_larceny 2014-12-31 23:00:00 314      7.266667   
91068  assault       2014-12-31 22:00:00 5        2.050000   
                                         7        2.066667   
                                         16       2.183333   
91082  theft_larceny 2014-12-31 22:00:00 395     11.916667   
91109  burglary      2014-12-31 14:30:00 36       9.900000   
91123  burglary      2014-12-31 12:30:00 33      11.866667   
91124  theft_larceny 2014-12-31 12:30:00 42      11.983333   
91140  assault       2015-01-01 16:15:00 605     -0.233333   
                                         629      0.316667   
91142  assault       2015-01-01 16:25:00 591     -0.750000   
                                         622      0.066667   
                                         715      2.366667   
                                         725      2.533333   
                                         770      3.350000   
                                         997     11.616667   
91145  burglary      2015-01-01 17:00:00 980      9.916667   
91147  burglary      2015-01-01 17:38:00 686      0.383333   
                                         705      0.883333   
91151  theft_larceny 2015-01-01 18:00:00 828      2.933333   
91153  assault       2015-01-01 16:30:00 638      0.200000   
...                                                    ...   
183932 theft_larceny 2016-12-31 14:00:00 633910   5.183333   
183935 assault       2016-12-31 13:30:00 633691   0.450000   
                                         633712   0.950000   
183936 theft_larceny 2016-12-31 13:30:00 633909   5.683333   
                                         633924   6.016667   
183937 theft_larceny 2016-12-31 13:00:00 633991   7.916667   
183938 robbery       2016-12-31 12:40:00 633644   0.000000   
183939 burglary      2016-12-31 12:31:00 633640   0.033333   
                                         633676   1.000000   
                                         633702   1.816667   
                                         633755   3.000000   
183940 robbery       2016-12-31 12:30:00 633648   0.283333   
183941 assault       2016-12-31 12:15:00 633625  -0.116667   
                                         633877   6.316667   
                                         633967   8.250000   
                                         633973   8.366667   
183942 robbery       2016-12-31 14:22:00 633710   0.033333   
183943 theft_larceny 2016-12-31 10:15:00 633708   4.133333   
183944 theft_larceny 2016-12-31 09:40:00 633569   0.416667   
                                         633641   2.933333   
183945 theft_larceny 2016-12-31 09:51:00 633619   2.083333   
183946 theft_larceny 2016-12-31 10:07:00 633588   0.833333   
183950 theft_larceny 2016-12-31 11:00:00 633608   0.650000   
                                         633652   1.900000   
                                         634021  10.766667   
183952 theft_larceny 2016-12-31 11:30:00 633587  -0.550000   
                                         633703   2.850000   
183953 assault       2016-12-31 11:45:00 633613  -0.033333   
                                         633923   7.750000   
183955 theft_larceny 2016-12-31 11:53:00 633653   1.050000   

                                                         DateTime_9  \
icrime Category_c    DateTime_c          i911                         
91

# plot on map

In [11]:

from importlib import reload
import src.vis; reload(src.vis)
import src.vis.map; reload(src.vis.map)
from src.vis.map import marker_cluster, marker_cluster_fast


In [17]:
from wKit.vis.map import marker_cluster_fast

In [13]:
name_location = {
    'crimes': list(zip(target_c.Latitude.tolist(), target_c.Longitude.tolist())),
    '911': list(zip(target_9.Latitude.tolist(), target_9.Longitude.tolist()))
}

In [20]:
marker_cluster_fast(name_location, lonlat=False, filename='tmp/crime_911.html')